In [60]:
# Time duration between contiguous registries
function get_duration(timec, datec, is_time_american)
    date_format = "dd/mm/yyyy HH:MM:SS"
    if(is_time_american)
        date_format = "mm/dd/yyyy HH:MM:SS"
    end
    time1 = Dates.DateTime(datec[2]*" "*string(timec[2]), date_format)
    time2 = Dates.DateTime(datec[1]*" "*string(timec[1]), date_format)
    return (abs((time2-time1).value)/1000)
end

function process_event(event_camion)
    event_id = "O"
    if(event_camion == "Encendido")
        event_id = "E"
    else
        if(event_camion == "Apagado")
            event_id = "A"
        end
    end
    return event_id
end

# Event Vector Formulation formatted as custom String
function get_events_string(events_vector)
    vector_str = "["
    for i in 1:(length(events_vector)-1)
        vector_str = vector_str * process_event(events_vector[i]) * ", "
    end
    return vector_str*process_event(events_vector[length(events_vector)])*"]"
end

# Generic Vector Formulation formatted as custom String
function get_vector_string(vector)
    vector_str = "["
    for i in 1:(length(vector)-1)
        vector_str = vector_str * string(vector[i]) * ", "
    end
    return vector_str*string(vector[length(vector)])*"]"
end

# Model Content obtention and Data recollection
function get_model_content(df_camion_filtered, limits, is_time_american, dParams)
    date_general = df_camion_filtered[!,:Fecha]
    time_general = df_camion_filtered[!,:Hora]
    
    time_camion = [get_duration([time_general[i],time_general[i+1]],[date_general[i],date_general[i+1]],is_time_american) for i in 1:length(time_general)-1]
    lat_camion = df_camion_filtered[!,:Lat]
    lon_camion = df_camion_filtered[!,:Lon]
    vel_camion = df_camion_filtered[!,:"Vel."]
    km_camion = df_camion_filtered[!,:Km]
    event_camion = df_camion_filtered[!,:Evento]

    model_str = " nreg = "*string(length(lat_camion))*"\n nparams = "*string(length(dParams))*"\n minstops = "*string(limits[1])*"\n maxstops = "*string(limits[2])*"\n time = "*get_vector_string(time_camion)*"\n lat = "*get_vector_string(lat_camion)*"\n lon = "*get_vector_string(lon_camion)*"\n vel = "*get_vector_string(vel_camion)*"\n km = "*get_vector_string(km_camion)*"\n events = "*string(get_events_string(event_camion))*"\n dParams = "*get_vector_string(dParams)
    println(model_str)
    return model_str
end

# Auxiliary function that saves a KML file
function save_dzn_file(filepath, str_to_save)
    open(filepath, "w") do dzn_file
        write(dzn_file, str_to_save)
    end
end

function minizinc_preprocessing(filenames::Vector{String}, limits::Vector{Int64}, is_time_american::Bool; params=dParams)

    model_count = 1
    for filepath in filenames
        df_camion = DataFrame(CSV.File(filepath))
        df_unique = unique(df_camion, 1)
        sort!(df_unique, :Fecha)
        unique_dates = df_unique[!,:Fecha]

        for date in unique_dates
            df_camion_filtered = filter(:Fecha => Fecha -> Fecha == date, df_camion)
            eachcol(df_camion_filtered)
            model_str = get_model_content(df_camion_filtered, limits, is_time_american, params)
            save_dzn_file("./src/Algorithms/StopsDeterm_MiniZinc/data/stop_determ_"*string(model_count)*".dzn", model_str)
            model_count = model_count + 1
        end
    end
end

minizinc_preprocessing (generic function with 2 methods)

In [19]:
[i+1 for i in 1:5]

5-element Vector{Int64}:
 2
 3
 4
 5
 6

In [7]:
using CSV

In [10]:
using DataFrames

In [24]:
using Dates

In [61]:

filenames = ["./data/TS_Stops_Camion_1B_01-08-2021.csv", "./data/TS_Stops_Camion_1B_01-09-2021.csv", "./data/TS_Stops_Camion_1B_02-08-2021.csv", "./data/TS_Stops_Camion_1B_02-09-2021.csv", "./data/TS_Stops_Camion_1B_03-08-2021.csv", "./data/TS_Stops_Camion_1B_03-09-2021.csv", "./data/TS_Stops_Camion_1B_04-08-2021.csv", "./data/TS_Stops_Camion_1B_04-09-2021.csv", "./data/TS_Stops_Camion_1B_05-09-2021.csv", "./data/TS_Stops_Camion_1B_06-08-2021.csv"]
minizinc_preprocessing(filenames, [80,120], true, params=[4,0.1,10,60,0.0001,1])


 nreg = 31
 nparams = 6
 minstops = 80
 maxstops = 120
 time = [3600.0, 3600.0, 3600.0, 3601.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 1905.0, 1695.0, 94.0, 3506.0, 899.0, 244.0, 2457.0, 1090.0, 2510.0, 3600.0, 57.0, 176.0, 3371.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0]
 lat = [-34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988]
 lon = [-56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632, -56.284632

 nreg = 66
 nparams = 6
 minstops = 80
 maxstops = 120
 time = [3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3603.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 848.0, 59.0, 15.0, 3.0, 2.0, 8.0, 5.0, 59.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 48.0, 573.0, 3600.0]
 lat = [-34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.869988, -34.870032, -34.87012, -34.870088, -34.86988, -34.869904, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.86992, -34.869

 nreg = 254
 nparams = 6
 minstops = 80
 maxstops = 120
 time = [3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 2588.0, 59.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0,

 nreg = 82
 nparams = 6
 minstops = 80
 maxstops = 120
 time = [3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0, 3575.0, 59.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 60.0, 38.0, 59.0, 60.0, 60.0, 30.0, 13.0, 786.0, 3600.0, 252.0, 741.0, 190.0, 2417.0, 3600.0, 23.0, 3577.0, 104.0, 3377.0, 119.0, 3600.0, 351.0, 3249.0, 3113.0, 128.0, 359.0, 3600.0, 3600.0, 3600.0, 3600.0, 3600.0]
 lat = [-34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.86988, -34.

In [71]:
function get_vector_for_matrix(vector)
    vector_str = ""
    for i in 1:(length(vector)-1)
        vector_str = vector_str * string(vector[i]) * ", "
    end
    return vector_str*string(vector[length(vector)])*"|"
end

function get_matrix_string(matrix)
    matrix_str = "["
    for i in 1:(size(matrix,1))
        matrix_str = matrix_str * get_vector_for_matrix(matrix[i,:]) * " "
    end
    return matrix_str*"]"
end

get_matrix_string (generic function with 1 method)

In [88]:
arr = [-1:0.0002:1;]
arr_tuple = Tuple(arr)

(-1.0, -0.9998, -0.9996, -0.9994, -0.9992, -0.999, -0.9988, -0.9986, -0.9984, -0.9982, -0.998, -0.9978, -0.9976, -0.9974, -0.9972, -0.997, -0.9968, -0.9966, -0.9964, -0.9962, -0.996, -0.9958, -0.9956, -0.9954, -0.9952, -0.995, -0.9948, -0.9946, -0.9944, -0.9942, -0.994, -0.9938, -0.9936, -0.9934, -0.9932, -0.993, -0.9928, -0.9926, -0.9924, -0.9922, -0.992, -0.9918, -0.9916, -0.9914, -0.9912, -0.991, -0.9908, -0.9906, -0.9904, -0.9902, -0.99, -0.9898, -0.9896, -0.9894, -0.9892, -0.989, -0.9888, -0.9886, -0.9884, -0.9882, -0.988, -0.9878, -0.9876, -0.9874, -0.9872, -0.987, -0.9868, -0.9866, -0.9864, -0.9862, -0.986, -0.9858, -0.9856, -0.9854, -0.9852, -0.985, -0.9848, -0.9846, -0.9844, -0.9842, -0.984, -0.9838, -0.9836, -0.9834, -0.9832, -0.983, -0.9828, -0.9826, -0.9824, -0.9822, -0.982, -0.9818, -0.9816, -0.9814, -0.9812, -0.981, -0.9808, -0.9806, -0.9804, -0.9802, -0.98, -0.9798, -0.9796, -0.9794, -0.9792, -0.979, -0.9788, -0.9786, -0.9784, -0.9782, -0.978, -0.9778, -0.9776, -0.9774, 

Internal error: encountered unexpected error in runtime:
StackOverflowError()
intersect at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect_unionall_ at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect_unionall at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect_unionall_ at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect_unionall at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect at /Applications/Julia-1.6.app/Contents/Resources/julia/lib/julia/libjulia-internal.1.6.dylib (unknown line)
intersect_unionall_ at /Applications/Julia-1.6.app/

In [89]:
function get_vector_string(vector)
    vector_str = "["
    for i in 1:(length(vector)-1)
        vector_str = vector_str * string(vector[i]) * ", "
    end
    return vector_str*string(vector[length(vector)])*"]"
end

get_vector_string (generic function with 1 method)

In [119]:
lat = 34.5(ones(92))
for i in 1:length(lat)
    rand_num = rand(arr_tuple)
    lat[i] = lat[i] + rand_num
    lat[i] = round(lat[i], digits=3)
end
get_vector_string(lat)

"[34.437, 35.254, 34.404, 35.089, 34.502, 34.708, 33.816, 34.464, 34.916, 34.829, 33.88, 34.316, 35.186, 33.558, 35.464, 35.305, 35.062, 35.096, 33.79, 34.76, 34.091, 34.591, 34.198, 33.765, 35.116, 34.701, 33.998, 35.144, 33.827, 33.749, 34.92, 34.541, 33.916, 34.682, 35.046, 33.545, 34.718, 34.552, 35.437, 33.746, 34.799, 33.965, 33.795, 34.848, 34.737, 34.522, 34.607, 34.333, 33.899, 34.961, 34.453, 33.64, 35.036, 35.251, 34.594, 35.117, 33.638, 34.451, 35.432, 35.103, 33.974, 35.276, 35.401, 33.572, 34.277, 34.305, 35.437, 33.947, 34.327, 33.782, 35.444, 34.936, 34.853, 34.728, 35.485, 34.473, 35.403, 33.867, 34.027, 34.87, 33.548, 34.281, 35.039, 33.991, 35.21, 33.813, 35.317, 34.513, 33.541, 35.147, 34.896, 34.621]"

In [120]:
lon = 56.3(ones(92))
for i in 1:length(lon)
    rand_num = rand(arr_tuple)
    lon[i] = lon[i] + rand_num
    lon[i] = round(lon[i], digits=3)
end
get_vector_string(lon)

"[56.64, 55.981, 56.578, 55.543, 55.963, 56.465, 55.785, 57.106, 55.326, 55.384, 56.941, 57.144, 55.878, 56.529, 56.921, 55.399, 56.826, 55.999, 55.981, 55.32, 56.658, 55.846, 57.055, 55.611, 56.289, 56.919, 55.505, 56.806, 56.608, 55.883, 56.219, 56.172, 56.682, 55.876, 55.919, 56.217, 56.787, 56.451, 57.242, 56.755, 56.206, 56.661, 56.539, 55.891, 56.825, 56.5, 56.774, 56.151, 56.932, 57.165, 56.339, 55.399, 56.978, 56.392, 56.887, 55.56, 56.094, 56.823, 55.317, 56.033, 56.985, 56.742, 55.729, 56.942, 55.67, 56.301, 55.5, 56.976, 56.041, 55.937, 56.686, 55.718, 55.753, 56.548, 57.194, 56.85, 56.738, 56.418, 56.017, 55.72, 55.396, 56.773, 56.006, 57.254, 55.522, 57.125, 57.294, 55.469, 56.174, 56.543, 57.227, 57.229]"

In [121]:
matrix = 34.7*ones(92, 92)
for i in 1:size(matrix,1)
    for j in 1:size(matrix,2)
        rand_num = rand(arr_tuple)
        matrix[i,j] = matrix[i,j] + rand_num
        matrix[i,j] = round(matrix[i,j], digits=3)
    end
end
get_matrix_string(matrix)

"[35.039, 34.279, 33.748, 35.628, 35.094, 34.294, 34.779, 35.348, 34.674, 34.898, 33.856, 35.515, 34.014, 35.538, 33.746, 34.048, 35.432, 35.459, 34.847, 35.127, 35.45, 34.3, 34.7, 33.738, 34.342, 34.775, 33.812, 34.586, 34.383, 34.457, 35.687, 33.778, 35.661, 34.927, 34.78, 35.44, 34.168, 34.288, 35.054, 34.962, 35.566, 35.128, 33.95, 35.355, 35.692, 35.214, 34.548, 34.941, 34.002, 34.34, 34.026, 34.016, 34.51, 35.277, 35.411, 34.107, 35.42, 35.28, 34.504, 35.333, 33.806, 34.047, 34.004, 33.908, 34.302, 35.214, 34.84, 34.234, 34.811, 34.838, 35.077, 33.866, 33.805, 35.161, 34.772, 34.15, 34.027, 33.72, 35.13, 34.513, 34.662, 34.84, 35.119, 34.454, 34.762, 34.642, 34.332, 34.096, 34.953, 35.4, 34.592, 35.636| 34.383, 35.46, 33.715, 34.773, 34.844, 33.738, 35.012, 35.114, 34.664, 34.351, 35.416, 34.748, 35.513, 33.966, 35.085, 33.765, 33.977, 34.438, 35.591, 34.423, 34.382, 34.154, 35.277, 34.919, 35.478, 34.326, 35.034, 35.08, 35.413, 34.424, 35.193, 35.339, 35.52, 33.821, 34.065, 35.4